In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import linalg
from Mesh import Mesh

import matplotlib.pyplot as plt


In [2]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
m = Mesh('data/coil_1.2_MM.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

print('Computing Laplacian')
K, M = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = linalg.eigh(K,M)

Computing Laplacian
Computing eigen values


In [4]:
# send all relevant numpy arrays to torch tensors
K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)
X = torch.from_numpy(m.verts).to(device)
N = X.shape[0]


# 1. Start with stronger regularization
#epsilon = 1e-4

# Apply the regularization
#K_reg = K + epsilon * torch.eye(N, device=device)
#print(f"\nUsing epsilon={epsilon}, final condition number: {torch.linalg.cond(K_reg).item():.2e}")

#print("\n=== Matrix Normalization ===")
#K_scale = torch.norm(K_reg, p='fro')
#M_scale = torch.norm(M, p='fro')

#K = K_reg / K_scale
#M = M / K_scale

In [5]:
# in the paper we used 50 eigenvalues so set k to 50
k = 50

In [6]:
# Build the neural network that maps coordinates -> k outputs per node
class MLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=k, hidden=[64,64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h, dtype=torch.double))
            layers.append(nn.Tanh())
            last = h
        layers.append(nn.Linear(last, out_dim, dtype=torch.double))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# Instantiate model and optimizer
model = MLP().to(device)
# Initialize all layers (Xavier), final layer small
for name, p in model.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
for p in model.net[-1].parameters():  # last Linear
    if p.ndim == 2:
        nn.init.normal_(p, std=1e-3)
    else:
        nn.init.zeros_(p)

lr_start = 0.05
lr_end = 0.001
max_epochs = 10_000
print_every = 1_000
loss_history = []

optimizer = optim.Adam(model.parameters(), lr=0.01)
decay_factor = (lr_end / lr_start) ** (1 / max_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_factor)

In [ ]:
for epoch in range(1, max_epochs+1):
    optimizer.zero_grad()
    U = model(X)  # N x k

    # losses
    UMU = U.T @ (M @ U)        # k x k
    UKU = U.T @ (K @ U) 
    orth_loss = torch.norm(UMU - torch.eye(k, device=device), p='fro')**2
    eig_loss = torch.norm(UKU, p='fro')**2

    loss = eig_loss + orth_loss

    loss.backward()
    optimizer.step()
    scheduler.step()
    loss_history.append(loss.item())

    if epoch % print_every == 0 or epoch == 1:
        approx_vals = torch.diag(U.T @ (K @ U)).detach().cpu().numpy()
        print(
        f"Epoch {epoch}, "
        f"total loss={loss.item():.6f}, "
        f"eig_loss={eig_loss.item():.6f}, "
        f"orth_loss={orth_loss.item():.6f}"
    )


Epoch 1, total loss=2497.217700, eig_loss=0.000010, orth_loss=49.944354
Epoch 1000, total loss=2201.805288, eig_loss=5.767390, orth_loss=43.920758
Epoch 2000, total loss=2009.677305, eig_loss=16.129654, orth_loss=39.870953
Epoch 3000, total loss=1921.751906, eig_loss=24.566942, orth_loss=37.943699
Epoch 4000, total loss=1833.446380, eig_loss=41.898296, orth_loss=35.830962
Epoch 5000, total loss=1694.038486, eig_loss=72.249184, orth_loss=32.435786
Epoch 6000, total loss=1641.323055, eig_loss=81.010171, orth_loss=31.206258
Epoch 7000, total loss=1536.596362, eig_loss=117.722790, orth_loss=28.377471
Epoch 8000, total loss=1464.815133, eig_loss=140.231847, orth_loss=26.491666
Epoch 9000, total loss=1389.144278, eig_loss=167.379269, orth_loss=24.435300
Epoch 10000, total loss=1306.605024, eig_loss=200.447962, orth_loss=22.123141


In [8]:
np.set_printoptions(suppress=True, precision=6)

# ==== Final result ====
with torch.no_grad():
    U_final = model(X)
    UKU = U_final.T @ (K @ U_final)
    final_eigs = torch.diag(UKU).cpu().numpy()
    #mu, Wsmall = np.linalg.eigh(UKU.cpu().numpy())
    #mu = np.real(mu)
    # sort
    #idx = np.argsort(mu)
    #mu = mu[idx]

    # FINAL EVALUATION
    abs_error = np.abs(final_eigs - eigvals[:k])
    rel_error = abs_error / (np.abs(eigvals[:k]) + 1e-10)
    #print("\nLearned Ritz values (from U^T K U):", np.round(mu[:5], 6))
    #print("Reference eigenvalues (first k):   ", np.round(eigvals[:5], 6))
    print("=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    
    # Eigenvalue comparison
    print(f"\nEigenvalue Comparison (first 10 modes):")
    print(f"{'Mode':<6} {'Predicted':<12} {'Reference':<12} {'Abs Error':<12} {'Rel Error':<12}")
    print("-" * 66)
    for i in range(min(10, k)):
        print(f"{i+1:<6} {final_eigs[i]:<12.6f} {eigvals[i]:<12.6f} "
              f"{abs_error[i]:<12.6f} {rel_error[i]:<12.4%}")
    
    print(f"\nEigenvalue Comparison (last 10 modes):")
    print(f"{'Mode':<6} {'Predicted':<12} {'Reference':<12} {'Abs Error':<12} {'Rel Error':<12}")
    print("-" * 66)
    for i in range(max(0, k-10), k):
        print(f"{i+1:<6} {final_eigs[i]:<12.6f} {eigvals[i]:<12.6f} "
              f"{abs_error[i]:<12.6f} {rel_error[i]:<12.4%}")
    
    # Overall statistics
    print(f"\nOverall Statistics (all {k} modes):")
    print(f"  Mean Absolute Error:   {np.mean(abs_error):.6f}")
    print(f"  Mean Relative Error:   {np.mean(rel_error):.4%}")
    print(f"  Median Relative Error: {np.median(rel_error):.4%}")
    print(f"  Max Relative Error:    {np.max(rel_error):.4%}")
    print(f"  Modes with <5% error:  {np.sum(rel_error < 0.05)}/{k}")
    print(f"  Modes with <10% error: {np.sum(rel_error < 0.10)}/{k}")
    
    print("\n" + "=" * 80)

FINAL RESULTS

Eigenvalue Comparison (first 10 modes):
Mode   Predicted    Reference    Abs Error    Rel Error   
------------------------------------------------------------------
1      2.478222     0.000000     2.478222     2469570552646.7212%
2      0.771758     0.007574     0.764183     10089.3704% 
3      1.144325     0.030308     1.114017     3675.6628%  
4      1.149125     0.068146     1.080978     1586.2571%  
5      2.586748     0.121208     2.465541     2034.1406%  
6      2.687989     0.189243     2.498746     1320.3920%  
7      1.191597     0.272231     0.919366     337.7147%   
8      1.556120     0.370536     1.185584     319.9646%   
9      0.911725     0.483409     0.428315     88.6030%    
10     0.911606     0.611343     0.300263     49.1154%    

Eigenvalue Comparison (last 10 modes):
Mode   Predicted    Reference    Abs Error    Rel Error   
------------------------------------------------------------------
41     1.109715     7.422193     6.312478     85.0487%  